# Derivation of field at point equations

We use `sympy` to derive the integrands for calculating the field at a point due to the currents on a scatterer.

From Collins 1969, the electric and magnetic potentials due to electric and magnetic current distributions $\mathbf{J}$ and $\mathbf{M}$ at the origin are:

$$ \mathbf{A}^e = \frac{\mu}{4\pi} \int_S \mathbf{J} \frac{1}{R} e^{-ikR} ds $$

And for the magnetic potential

$$ \mathbf{A}^m = \frac{\epsilon}{4\pi} \int_S \mathbf{M} \frac{1}{R} e^{-ikR} ds $$


The electric and magnetic fields at a point are given by:

$$ \mathbf{E} = -i\omega \Big(\mathbf{A}^e + \frac{1}{k^2}\nabla\big(\nabla\cdot\mathbf{A}^e\big)\Big) - \frac{1}{\epsilon}\nabla\times\mathbf{A}^m $$

$$ \mathbf{H} = -i\omega \Big(\mathbf{A}^m + \frac{1}{k^2}\nabla\big(\nabla\cdot\mathbf{A}^m\big)\Big) + \frac{1}{\mu}\nabla\times\mathbf{A}^e $$

We calculate the field contribution from a single point on the scatterer surface - i.e. the integrand above - as:

$$ \mathbf{E}_n = \frac{-i\omega}{4\pi} \Bigg( \mu \Big( \mathbf{J}_n \frac{1}{R} e^{-ikR} + \frac{1}{k^2}\nabla\big(\nabla\cdot (\mathbf{J}_n \frac{1}{R} e^{-ikR}) \big) \Big) \Bigg) - \frac{1}{4\pi}\nabla\times(\mathbf{M}_n \frac{1}{R} e^{-ikR}) $$

$$ \mathbf{H}_n = \frac{-i\omega}{4\pi} \Bigg( \epsilon \Big(\mathbf{M}_n \frac{1}{R} e^{-ikR} + \frac{1}{k^2}\nabla\big(\nabla\cdot ( \mathbf{M}_n \frac{1}{R} e^{-ikR}) \big) \Big) \Bigg) + \frac{1}{4 \pi}\nabla\times ( \mathbf{J}_n \frac{1}{R} e^{-ikR}) $$

Where $\mathbf{J}_n$ and $\mathbf{M}_n$ are constant complex 3-vectors at a point on the scattering surface.

We now set up `sympy` to apply these differential operators.

In [2]:
from sympy import symbols, diff, exp, I, sqrt, separatevars, sin, cos
from sympy.vector import CoordSys3D, Del, curl, divergence, gradient, express
delop = Del()

In [3]:
X = CoordSys3D('X')
S = X.create_new('S', transformation='spherical')

In [4]:
pi, k, eps, mu, R, r= symbols("pi, k, epsilon, mu, R, r")

omega = k / sqrt(eps*mu)

Rs = (X.x*X.i + X.y*X.j + X.z*X.k)

r = Rs.magnitude()

Rhat = Rs.normalize()

jx, jy, jz = symbols("j_x, j_y, j_z", complex=True)

Js = jx*X.i + jy*X.j + jz*X.k

J, M = symbols('J, M')

mx, my, mz = symbols("m_x, m_y, m_z", complex=True)

Ms = mx*X.i + my*X.j + mz*X.k

Z = sqrt(mu/eps)

In [5]:
Rhat.subs(r, R)

X.x/R*X.i + X.y/R*X.j + X.z/R*X.k

# Green's Function

We define the freespace Green's function $G$ as:

$$ G = \frac{e^{-i k R}}{R} $$

In [6]:
G = exp(-I*k*r) / r

# Vector Potentials

In [7]:
A_e = mu / (4*pi) * Js * G
A_e.subs(r, R)

(j_x*mu*exp(-I*R*k)/(4*R*pi))*X.i + (j_y*mu*exp(-I*R*k)/(4*R*pi))*X.j + (j_z*mu*exp(-I*R*k)/(4*R*pi))*X.k

In [8]:
A_m = eps / (4*pi) * Ms * G
A_m.subs(r, R)

(epsilon*m_x*exp(-I*R*k)/(4*R*pi))*X.i + (epsilon*m_y*exp(-I*R*k)/(4*R*pi))*X.j + (epsilon*m_z*exp(-I*R*k)/(4*R*pi))*X.k

# First Terms
The first terms in each integrand are simple.


## First E-field Term
$$ \mathbf{E}_1 = -i \omega \mathbf{A}^e $$

In [9]:
# Calculate each term
E_1 = -I*omega*A_e
E_1 = express(E_1, X, variables=True)
E_1.subs(r, R)

(-I*j_x*k*mu*exp(-I*R*k)/(4*R*pi*sqrt(epsilon*mu)))*X.i + (-I*j_y*k*mu*exp(-I*R*k)/(4*R*pi*sqrt(epsilon*mu)))*X.j + (-I*j_z*k*mu*exp(-I*R*k)/(4*R*pi*sqrt(epsilon*mu)))*X.k

### Result
By inspection:

$$\mathbf{E}_1 = \frac{Z}{4\pi} k^2 \mathbf{J} \Bigg(\frac{-i}{k R}\Bigg) e^{-i k R} $$

## First H-field Term

$$ \mathbf{H}_1 = -i \omega \mathbf{A}^m $$

In [10]:
H_1 = -I*omega*A_m
H_1 = express(H_1, X, variables=True).subs(r, R)
H_1

(-I*epsilon*k*m_x*exp(-I*R*k)/(4*R*pi*sqrt(epsilon*mu)))*X.i + (-I*epsilon*k*m_y*exp(-I*R*k)/(4*R*pi*sqrt(epsilon*mu)))*X.j + (-I*epsilon*k*m_z*exp(-I*R*k)/(4*R*pi*sqrt(epsilon*mu)))*X.k

### Result

Again by inspection:

$$\mathbf{H}_1 = \frac{1}{4\pi Z} k^2 \mathbf{M} \Bigg(\frac{-i}{k R}\Bigg) e^{-i k R} $$

# Second Terms

he second terms are more complex:

## Second E-field Term

$$ \mathbf{E}_2 = \frac{-i \omega}{k^2} \nabla \big(\nabla\cdot\mathbf{A}^e \big) $$

This is less obvious, but we can make use of the following results:

In [11]:
Js_Rh_Rh_X = (Js.dot(Rhat)*Rhat).expand()
Js_Rh_Rh_X.subs(r, R)

(X.x**2*j_x/R**2 + X.x*X.y*j_y/R**2 + X.x*X.z*j_z/R**2)*X.i + (X.x*X.y*j_x/R**2 + X.y**2*j_y/R**2 + X.y*X.z*j_z/R**2)*X.j + (X.x*X.z*j_x/R**2 + X.y*X.z*j_y/R**2 + X.z**2*j_z/R**2)*X.k

In [13]:
E_2 = -I * omega / k**2 * gradient(divergence(A_e))

E_2.dot(X.i).subs(r, R).expand()

X.x**2*I*j_x*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x**2*j_x*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x**2*I*j_x*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.x*X.y*I*j_y*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*j_y*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*j_y*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.x*X.z*I*j_z*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*j_z*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*j_z*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - j_x*mu*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*j_x*mu*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [16]:
E_2y = E_2.dot(X.j).doit()
E_2y.subs(r, R).expand()

X.x*X.y*I*j_x*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*j_x*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*j_x*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y**2*I*j_y*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y**2*j_y*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y**2*I*j_y*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y*X.z*I*j_z*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*j_z*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*j_z*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - j_y*mu*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*j_y*mu*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [17]:
E_2z = E_2.dot(X.k).doit()
E_2z.subs(r, R).expand()

X.x*X.z*I*j_x*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*j_x*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*j_x*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y*X.z*I*j_y*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*j_y*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*j_y*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.z**2*I*j_z*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.z**2*j_z*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.z**2*I*j_z*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - j_z*mu*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*j_z*mu*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

### Result

By grouping common factors and comparing each term in turn, we arrive at:

$$ \mathbf{E}_{2} = \frac{i k^2 Z }{4 \pi} \Bigg[(\mathbf{J}\cdot\mathbf{\hat{R}})\mathbf{\hat{R}}\Bigg( \frac{i}{kR} + \frac{3}{(kR)^2} - \frac{3i}{(kR)^3} \Bigg) + \mathbf{J} \Bigg(\frac{-1}{(kR)^2} + \frac{i}{(kR)^3}\Bigg)\Bigg] e^{-i k R} $$

We check this result below:

In [18]:
prefactor = k**2 * mu * exp(-I*k*r) / (4*pi) / sqrt(eps*mu)
prefactor.subs(r, R)

k**2*mu*exp(-I*R*k)/(4*pi*sqrt(epsilon*mu))

In [19]:
kR_sum_2 = (I/(k*r) + 3/(k*r)**2 - 3*I/(k*r)**3)

kR_sum_2.subs(r, R)

I/(R*k) + 3/(R**2*k**2) - 3*I/(R**3*k**3)

In [32]:
Trial = prefactor*(kR_sum_2*Js_Rh_Rh_X.expand() + (-1/(k*R)**2 + I/(k*R)**3)*Js)
Trial.dot(X.i).subs(r,R).expand()

X.x**2*I*j_x*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x**2*j_x*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x**2*I*j_x*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.x*X.y*I*j_y*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*j_y*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*j_y*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.x*X.z*I*j_z*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*j_z*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*j_z*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - j_x*mu*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*j_x*mu*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [33]:
Trial.dot(X.j).subs(r,R).expand()

X.x*X.y*I*j_x*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*j_x*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*j_x*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y**2*I*j_y*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y**2*j_y*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y**2*I*j_y*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y*X.z*I*j_z*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*j_z*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*j_z*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - j_y*mu*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*j_y*mu*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [34]:
Trial.dot(X.k).subs(r,R).expand()

X.x*X.z*I*j_x*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*j_x*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*j_x*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y*X.z*I*j_y*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*j_y*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*j_y*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.z**2*I*j_z*k*mu*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.z**2*j_z*mu*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.z**2*I*j_z*mu*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - j_z*mu*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*j_z*mu*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

## Second H-field Term

$$ \mathbf{H}_2 = \frac{-i \omega}{k^2} \nabla \big(\nabla\cdot\mathbf{A}^m \big) $$

In [37]:
H_2 = -I * omega / k**2 * gradient(divergence(A_m))

H_2.dot(X.i).subs(r, R).expand()

X.x**2*I*epsilon*k*m_x*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x**2*epsilon*m_x*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x**2*I*epsilon*m_x*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.x*X.y*I*epsilon*k*m_y*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*epsilon*m_y*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*epsilon*m_y*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.x*X.z*I*epsilon*k*m_z*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*epsilon*m_z*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*epsilon*m_z*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - epsilon*m_x*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*epsilon*m_x*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [51]:
H_2.dot(X.j).subs(r, R).expand()

X.x*X.y*I*epsilon*k*m_x*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*epsilon*m_x*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*epsilon*m_x*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y**2*I*epsilon*k*m_y*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y**2*epsilon*m_y*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y**2*I*epsilon*m_y*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y*X.z*I*epsilon*k*m_z*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*epsilon*m_z*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*epsilon*m_z*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - epsilon*m_y*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*epsilon*m_y*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [52]:
H_2.dot(X.k).subs(r, R).expand()

X.x*X.z*I*epsilon*k*m_x*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*epsilon*m_x*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*epsilon*m_x*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.y*X.z*I*epsilon*k*m_y*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*epsilon*m_y*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*epsilon*m_y*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) + X.z**2*I*epsilon*k*m_z*exp(-I*R*k)/(4*R**3*pi*sqrt(epsilon*mu)) + 3*X.z**2*epsilon*m_z*exp(-I*R*k)/(4*R**4*pi*sqrt(epsilon*mu)) - 3*X.z**2*I*epsilon*m_z*exp(-I*R*k)/(4*R**5*k*pi*sqrt(epsilon*mu)) - epsilon*m_z*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*epsilon*m_z*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

### Result

$$ \mathbf{H}_{2} = \frac{i k^2 Z }{4 \pi} \Bigg[(\mathbf{M}\cdot\mathbf{\hat{R}})\mathbf{\hat{R}}\Bigg( \frac{i}{kR} + \frac{3}{(kR)^2} - \frac{3i}{(kR)^3} \Bigg) + \mathbf{M} \Big(\frac{-1}{(kR)^2} + \frac{i}{(kR)^3}\Bigg)\Bigg] e^{-i k R} $$

We check this result below:

In [40]:
Ms_Rh_Rh_X = Ms.dot(Rhat)*Rhat
Ms_Rh_Rh_X_y = Ms_Rh_Rh_X.dot(X.j)
Ms_Rh_Rh_X_y.expand().subs(r, R)

X.x*X.y*m_x/R**2 + X.y**2*m_y/R**2 + X.y*X.z*m_z/R**2

In [45]:
prefactor = k**2 * eps * exp(-I*k*r) / (4*pi) / sqrt(eps*mu)
prefactor.subs(r, R)

epsilon*k**2*exp(-I*R*k)/(4*pi*sqrt(epsilon*mu))

In [46]:
Trial = prefactor*(1/(Z**2)*kR_sum_2*Ms_Rh_Rh_X.expand() + (-1/(k*R)**2 + I/(k*R)**3)*Ms)

Trial.dot(X.i).subs(r,R).expand()

X.x**2*I*epsilon**2*k*m_x*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.x**2*epsilon**2*m_x*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.x**2*I*epsilon**2*m_x*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) + X.x*X.y*I*epsilon**2*k*m_y*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*epsilon**2*m_y*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*epsilon**2*m_y*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) + X.x*X.z*I*epsilon**2*k*m_z*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.x*X.z*epsilon**2*m_z*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.x*X.z*I*epsilon**2*m_z*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) - epsilon*m_x*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*epsilon*m_x*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [49]:
Trial.dot(X.j).subs(r,R).expand()

X.x*X.y*I*epsilon**2*k*m_x*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*epsilon**2*m_x*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*epsilon**2*m_x*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) + X.y**2*I*epsilon**2*k*m_y*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.y**2*epsilon**2*m_y*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.y**2*I*epsilon**2*m_y*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) + X.y*X.z*I*epsilon**2*k*m_z*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*epsilon**2*m_z*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*epsilon**2*m_z*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) - epsilon*m_y*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*epsilon*m_y*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

In [50]:
Trial.dot(X.j).subs(r,R).expand()

X.x*X.y*I*epsilon**2*k*m_x*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.x*X.y*epsilon**2*m_x*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.x*X.y*I*epsilon**2*m_x*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) + X.y**2*I*epsilon**2*k*m_y*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.y**2*epsilon**2*m_y*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.y**2*I*epsilon**2*m_y*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) + X.y*X.z*I*epsilon**2*k*m_z*exp(-I*R*k)/(4*R**3*mu*pi*sqrt(epsilon*mu)) + 3*X.y*X.z*epsilon**2*m_z*exp(-I*R*k)/(4*R**4*mu*pi*sqrt(epsilon*mu)) - 3*X.y*X.z*I*epsilon**2*m_z*exp(-I*R*k)/(4*R**5*k*mu*pi*sqrt(epsilon*mu)) - epsilon*m_y*exp(-I*R*k)/(4*R**2*pi*sqrt(epsilon*mu)) + I*epsilon*m_y*exp(-I*R*k)/(4*R**3*k*pi*sqrt(epsilon*mu))

Comparing term by term, this is the same as the result above

# Third Terms

The third terms are:

## Third E-field Term

$$ \mathbf{E}_3 = \frac{1}{\epsilon} \nabla \times A_m $$

In [601]:
Ms.cross(Rhat).subs(r, R)

(-X.y*m_z/R + X.z*m_y/R)*X.i + (X.x*m_z/R - X.z*m_x/R)*X.j + (-X.x*m_y/R + X.y*m_x/R)*X.k

In [598]:
E_3 = - curl(A_m) / eps

E_3.dot(X.i).subs(r, R).expand()

X.y*I*k*m_z*exp(-I*R*k)/(4*R**2*pi) + X.y*m_z*exp(-I*R*k)/(4*R**3*pi) - X.z*I*k*m_y*exp(-I*R*k)/(4*R**2*pi) - X.z*m_y*exp(-I*R*k)/(4*R**3*pi)

In [602]:
E_3.dot(X.j).subs(r, R).expand()

-X.x*I*k*m_z*exp(-I*R*k)/(4*R**2*pi) - X.x*m_z*exp(-I*R*k)/(4*R**3*pi) + X.z*I*k*m_x*exp(-I*R*k)/(4*R**2*pi) + X.z*m_x*exp(-I*R*k)/(4*R**3*pi)

In [603]:
E_3.dot(X.k).subs(r, R).expand()

X.x*I*k*m_y*exp(-I*R*k)/(4*R**2*pi) + X.x*m_y*exp(-I*R*k)/(4*R**3*pi) - X.y*I*k*m_x*exp(-I*R*k)/(4*R**2*pi) - X.y*m_x*exp(-I*R*k)/(4*R**3*pi)

### Result


By inspection:

$$ \mathbf{E}_3 = - \frac{k^2}{4\pi} \mathbf{M} \times \mathbf{\hat{R}} \Bigg(\frac{i}{kR}+\frac{1}{(kR)^2}\Bigg) e^{-i k R} $$

## Third H-field Term

$$ \mathbf{H}_3 = \frac{1}{\mu} \nabla \times A_e $$

In [605]:
H_3 = curl(A_e) / mu
H_3.dot(X.i).subs(r, R).expand()


-X.y*I*j_z*k*exp(-I*R*k)/(4*R**2*pi) - X.y*j_z*exp(-I*R*k)/(4*R**3*pi) + X.z*I*j_y*k*exp(-I*R*k)/(4*R**2*pi) + X.z*j_y*exp(-I*R*k)/(4*R**3*pi)

In [606]:
H_3.dot(X.j).subs(r, R).expand()

X.x*I*j_z*k*exp(-I*R*k)/(4*R**2*pi) + X.x*j_z*exp(-I*R*k)/(4*R**3*pi) - X.z*I*j_x*k*exp(-I*R*k)/(4*R**2*pi) - X.z*j_x*exp(-I*R*k)/(4*R**3*pi)

In [607]:
H_3.dot(X.k).subs(r, R).expand()

-X.x*I*j_y*k*exp(-I*R*k)/(4*R**2*pi) - X.x*j_y*exp(-I*R*k)/(4*R**3*pi) + X.y*I*j_x*k*exp(-I*R*k)/(4*R**2*pi) + X.y*j_x*exp(-I*R*k)/(4*R**3*pi)

### Result
By inspection:

$$ \mathbf{H}_3 = \frac{k^2}{4\pi} \mathbf{J} \times \mathbf{\hat{R}} \Bigg(\frac{i}{kR}+\frac{1}{(kR)^2}\Bigg) e^{-i k R} $$

# Final Form

We put together the three terms for each field to obtain:

$$ \mathbf{E} = \mathbf{E}_1 + \mathbf{E}_2 + \mathbf{E}_3 $$
$$ \mathbf{E} = \frac{Z}{4\pi} k^2 \mathbf{J} \Bigg(\frac{-i}{k R}\Bigg) e^{-i k R} $$
$$ + \frac{k^2 Z }{4 \pi} \Bigg[(\mathbf{J}\cdot\mathbf{\hat{R}})\mathbf{\hat{R}}\Bigg( \frac{i}{kR} + \frac{3}{(kR)^2} - \frac{3i}{(kR)^3} \Bigg) + \mathbf{J} \Bigg(\frac{-1}{(kR)^2} + \frac{i}{(kR)^3}\Bigg)\Bigg] e^{-i k R} $$
$$ - \frac{k^2}{4\pi} \mathbf{M} \times \mathbf{\hat{R}} \Bigg(\frac{i}{kR}+\frac{1}{(kR)^2}\Bigg) e^{-i k R} $$

## E-field

$$ \mathbf{E} = \frac{k^2}{4\pi} \Bigg\{ Z \Bigg[J \Bigg( \frac{-i}{kR} - \frac{1}{(kR)^2} + \frac{i}{(kR)^3}\Bigg) + (\mathbf{J}\cdot\mathbf{\hat{R}})\mathbf{\hat{R}}\Bigg( \frac{i}{kR} + \frac{3}{(kR)^2} - \frac{3i}{(kR)^3} \Bigg)\Bigg] $$

$$ - \mathrm{M}\times\mathrm{\hat{R}}\Bigg(\frac{i}{kR}+\frac{1}{(kR)^2}\Bigg)\Bigg\} e^{-ikR} $$

$$ \mathbf{H} = \mathbf{H}_1 + \mathbf{H}_2 + \mathbf{H}_3 $$
$$ \mathbf{H} = \frac{1}{4\pi Z} k^2 \mathbf{M} \Bigg(\frac{-i}{k R}\Bigg) e^{-i k R} $$
$$ + \frac{k^2}{4 \pi Z} \Bigg[(\mathbf{M}\cdot\mathbf{\hat{R}})\mathbf{\hat{R}}\Bigg( \frac{i}{kR} + \frac{3}{(kR)^2} - \frac{3i}{(kR)^3} \Bigg) + \mathbf{M} \Big(\frac{-1}{(kR)^2} + \frac{i}{(kR)^3}\Bigg)\Bigg] e^{-i k R} $$
$$ + \frac{k^2}{4\pi} \mathbf{J} \times \mathbf{\hat{R}} \Bigg(\frac{i}{kR}+\frac{1}{(kR)^2}\Bigg) e^{-i k R} $$

## H-field

$$ \mathbf{H} = \frac{k^2}{4\pi} \Bigg\{ \frac{1}{Z} \Bigg[M \Bigg( \frac{-i}{kR} - \frac{1}{(kR)^2} + \frac{i}{(kR)^3}\Bigg) + (\mathbf{M}\cdot\mathbf{\hat{R}})\mathbf{\hat{R}}\Bigg( \frac{i}{kR} + \frac{3}{(kR)^2} - \frac{3i}{(kR)^3} \Bigg)\Bigg] $$

$$ + \mathrm{J}\times\mathrm{\hat{R}}\Bigg(\frac{i}{kR}+\frac{1}{(kR)^2}\Bigg)\Bigg\} e^{-ikR} $$

# Farfields

For propagation in the forward direction, the far field is defined by:

$$\mathbf{E}_{far} = \lim_{r\rightarrow\infty} \mathbf{E}(\mathbf{r}) k r e^{ikr}$$

$$\mathbf{H}_{far} = \lim_{r\rightarrow\infty} \mathbf{H}(\mathbf{r}) k r e^{ikr}$$

where $r$ = $|\mathbf{r}|$

Inserting the above equations gives:

$$ \mathbf{E}_{far} = -\frac{ik^2}{4\pi} \int_S \Bigg(Z \Big(\mathbf{J} - (\mathbf{J}\cdot\mathbf{\hat{r}})\mathbf{\hat{r}}\Big) - \mathbf{\hat{r}} \times \mathbf{M} \Bigg) e^{ik\mathbf{r}_s\cdot\mathbf{\hat{r}}} ds $$


$$ \mathbf{H}_{far} = -\frac{ik^2}{4\pi} \int_S \Bigg(\frac{1}{Z} \Big(\mathbf{M} - (\mathbf{M}\cdot\mathbf{\hat{r}})\mathbf{\hat{r}}\Big) + \mathbf{\hat{r}} \times \mathbf{J} \Bigg) e^{ik\mathbf{r}_s\cdot\mathbf{\hat{r}}} ds $$

In [ ]:
Inserting the 